In [1]:
import requests
import json
import os

from dotenv import load_dotenv
from pathlib import Path

from web3.auto import w3
load_dotenv()

True

In [2]:
headers = {
    "Content-Type": "application/json",
    "pinata_api_key": os.getenv("PINATA_API_KEY"),
    "pinata_secret_api_key": os.getenv("PINATA_SECRET_API_KEY"),
}

In [3]:
def initContract():
    with open(Path("courthouse.json")) as json_file:
        abi = json.load(json_file)

    return w3.eth.contract(address=os.getenv("COURTHOUSE_ADDRESS"), abi=abi)

In [4]:
def convertDataToJSON(address, square_footage, build_date,property_type):
    data = {
        "pinataOptions": {"cidVersion": 1},
        "pinataContent": {
            "PROPERTY_ADDRESS": address,
            "SQ.FT": square_footage,
            "BUILD_DATE": build_date,
            "TYPE": property_type,
        },
    }
    return json.dumps(data)

In [5]:
def pinJSONtoIPFS(json):
    r = requests.post(
        "https://api.pinata.cloud/pinning/pinJSONToIPFS", data=json, headers=headers
    )
    ipfs_hash = r.json()["IpfsHash"]
    return f"ipfs://{ipfs_hash}"

In [6]:
def generateIPFS_uri():
    property_address = input("What is the address?: ")
    property_size = input("What is the square footage?: ")
    property_age = input("When was the property built?: ")
    property_type = input("Enter legal description of property: ")

    json_data = convertDataToJSON(property_address,
                                property_size,
                                property_age,
                                property_type)
    uri = pinJSONtoIPFS(json_data)

    return uri

In [7]:
courthouse = initContract()

In [8]:
def tokenize_property():
    owner_wallet_address = input("Token holders wallet address: ")
    property_address = input("What is the address?: ")
    property_type = input("Enter legal description of property: ")
    property_size = input("What is the square footage?: ")
    property_age = input("When was the property built?: ")
    property_last_price = 0
    json_data = convertDataToJSON(property_address,
                                property_size,
                                property_age,
                                property_type)
    uri = pinJSONtoIPFS(json_data)
    tx_hash = courthouse.functions.tokenizeProperty(owner_wallet_address, property_address, property_type, property_last_price, uri).transact(
        {"from": w3.eth.accounts[2]}
    )
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    print(uri)
    return receipt

In [12]:
token = tokenize_property()

Token holders wallet address:  0x77d959E069A5Ab69BdA377C6499B6EAfee165D10
What is the address?:  123 main
Enter legal description of property:  single
What is the square footage?:  30
When was the property built?:  1980


ContractLogicError: execution reverted: VM Exception while processing transaction: revert uri is not unique

In [10]:
token

AttributeDict({'transactionHash': HexBytes('0xa81d2ed21e31d657dc3caa7da3bba2508ee9652a90aae6346f3512b4e145c941'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x215eae5878fcd4f5fd47e1f892465c901f12804a47cf5a8a6482dcdd90de7ffc'),
 'blockNumber': 415,
 'from': '0x77d959E069A5Ab69BdA377C6499B6EAfee165D10',
 'to': '0xEbBa2E5B09151A020Bf42852DEe399DaeB31279F',
 'gasUsed': 487127,
 'cumulativeGasUsed': 487127,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0xa81d2ed21e31d657dc3caa7da3bba2508ee9652a90aae6346f3512b4e145c941'),
   'blockHash': HexBytes('0x215eae5878fcd4f5fd47e1f892465c901f12804a47cf5a8a6482dcdd90de7ffc'),
   'blockNumber': 415,
   'address': '0xEbBa2E5B09151A020Bf42852DEe399DaeB31279F',
   'data': '0x',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x00000